# Hand Gesture Detection using YOLOv8
A comprehensive guide to implementing real-time hand gesture detection using YOLOv8 and Roboflow.

## Table of Contents
1. [Introduction](#introduction)
2. [Setup and Installation](#setup-and-installation)
3. [Dataset Preparation](#dataset-preparation)
4. [Model Training](#model-training)
5. [Detection Functions](#detection-functions)
   - [Image Detection](#image-detection)
   - [Real-time Detection](#real-time-detection)
6. [Usage Examples](#usage-examples)


## Introduction
This project implements a hand gesture detection system using YOLOv8 and Roboflow. It can perform real-time detection through a webcam feed and process individual images. The system can also save detection results as video files for later analysis.


## Setup and Installation
First, install the required packages using pip:

In [ ]:
%pip install ultralytics roboflow

Import the necessary libraries:

In [ ]:
from roboflow import Roboflow
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2 as cv
import os

## Dataset Preparation
Connect to Roboflow and download the dataset:

In [ ]:
# Initialize Roboflow connection
# Replace YOUR_API_KEY with your actual Roboflow API key
rf = Roboflow(api_key="YOUR_API_KEY")

# Access the project and download dataset
project = rf.workspace("puspendu-ai-vision-workspace").project("hand_gesture_detection-xdcpy")
version = project.version(1)
dataset = version.download("yolov8")

> **Note**: Make sure to replace `YOUR_API_KEY` with your actual Roboflow API key. You can obtain this from your Roboflow dashboard.

## Model Training
Train the YOLOv8 model using the downloaded dataset:

In [ ]:
# Initialize YOLO model
model = YOLO("yolov8l.pt")

# Train the model
# Note: Update the data.yaml path with your absolute path
model.train(
    data="HAND_GESTURE_DETECTION-1/data.yaml",
    epochs=50,
    imgsz=640
)

> **Important**: 
> - The `data.yaml` file path should be updated with the absolute path of your dataset
> - Training time will vary based on your hardware capabilities
> - Using GPU acceleration is recommended for faster training

## Detection Functions

### Image Detection
Function to display predictions on single images:

In [ ]:
def display_prediction(image_path):
    """
    Display detection results for a single image.
    
    Args:
        image_path (str): Path to the input image
        
    Returns:
        None: Displays the image with detection results
    """
    trained_model = YOLO("runs/detect/train/weights/best.pt")
    result = trained_model.predict(image_path, conf=0.5)
    img_rgb = cv.cvtColor(result[0].plot(), cv.COLOR_BGR2RGB)
    
    fig = plt.figure(figsize=(12, 10))
    plt.imshow(img_rgb)
    plt.axis("off")
    plt.show()

### Real-time Detection
Enhanced function for real-time webcam detection with video saving capability:

In [ ]:
def run_detection(conf=0.5, save_video=False, filename=None):
    """
    Run real-time object detection on webcam feed using YOLOv8 model.

    Parameters:
    - conf: Confidence threshold for detections (default: 0.5)
    - save_video: Flag to save output as a video file (default: False)
    - filename: Optional filename for saved video. If None, an incremented filename is generated.
    """
    # Load the model
    try:
        model = YOLO("runs/detect/train/weights/best.pt")
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    # Handle filename for saving video
    if save_video:
        if filename is None:
            # Generate an auto-incremented filename if none provided
            count = 1
            while os.path.exists(f"VIDEOS/detection_result_{count}.mp4"):
                count += 1
            filename = f"VIDEOS/detection_result_{count}.mp4"
        else:
            # Ensure .mp4 extension if user specifies filename
            if not filename.endswith(".mp4"):
                filename += ".mp4"
            filename = f"VIDEOS/{filename}"

        print(f"Recording video to {filename}")

    # Start video capture
    cap = cv.VideoCapture(0)
    if not cap.isOpened():
        print("Error: couldn't open the webcam!")
        return

    # Capture frame width, height, and set fps
    w, h = (int(cap.get(x)) for x in (3, 4))
    fps = 15
    out = None

    # Set up video writer if saving is enabled
    if save_video:
        out = cv.VideoWriter(filename, cv.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    print("Press 's' to start saving, 'p' to pause and exit.")

    # Main loop to read frames and perform detection
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: couldn't read a frame from the webcam.")
            break

        # Flip the frame horizontally
        frame = cv.flip(frame, 1)

        # Run YOLO model prediction
        try:
            result = model.predict(frame, conf=conf, verbose=False)[0]
            frame = result.plot()
        except Exception as e:
            print(f"Error during model prediction: {e}")
            break

        # Display the frame
        cv.imshow("Camera", frame)

        # Handle user input
        key = cv.waitKey(1) & 0xFF
        if key == ord('s') and save_video:
            print("Video recording started.")
        elif key == ord('p'):
            print("Stopping and closing the program.")
            break

        # Save frame if enabled
        if save_video and out is not None:
            out.write(frame)

    # Release resources
    cap.release()
    if out:
        out.release()
    cv.destroyAllWindows()
    print("Program ended and resources released.")




> **Features**:
> - Real-time detection through webcam
> - Adjustable confidence threshold
> - Optional video saving
> - Horizontal frame flipping for natural interaction
> - Error handling and resource management
> - Simple keyboard controls ('s' to start saving, 'p' to pause/exit)

## Usage Examples

1. Run real-time detection without saving:

In [ ]:
run_detection(conf=0.5)

2. Run detection and save video with auto-generated filename:

In [ ]:
run_detection(conf=0.5, save_video=True)

3. Run detection and save video with custom filename:

In [ ]:
run_detection(conf=0.5, save_video=True, filename="my_detection")

4. Process a single image:

In [ ]:
display_prediction("path/to/your/image.jpg")

> **Best Practices**:
> - Ensure good lighting conditions for better detection
> - Keep hand gestures within the camera frame
> - Try to capture videos with a background as shown in the test images
> - While recording videos try to show only your hands not faces as the model is trained only with hand images
> - Adjust confidence threshold based on your needs
> - Create a 'VIDEOS' directory before saving recordings
> - Regularly backup your trained models


## Project link
 - You can find the project on Roboflow by clicking [here](https://universe.roboflow.com/puspendu-ai-vision-workspace/hand_gesture_detection-xdcpy)
 - You can find the training logs on on wandb by clicking on this [link](https://wandb.ai/ranapuspendu24-iit-madras-foundation/Ultralytics/runs/zayjstbb?nw=nwuserranapuspendu24)